In [22]:
import os
import os
import numpy as np 
import scipy as sp 
import matplotlib as mpl 
import matplotlib.cm as cm 
import matplotlib.pyplot as plt 
import pandas as pd 
import nltk
import re
import csv
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix
from time import time

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

%matplotlib inline


In [23]:
def swn(textlist):
    """
    given list of words, return a list of the sentiment value of each word
    """
    from nltk.corpus import sentiwordnet

    # first tag all the words with appropriate pos
    poslist = nltk.pos_tag(textlist)
    swnvalues = list()
    for w in poslist:
        postagged = 0
        if 'NN' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'n'))
        elif 'JJ' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'a'))
        elif 'RB' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'r'))
        elif 'VB' in w[1]:
            word = list(sentiwordnet.senti_synsets(w[0],'v'))
        else:
            postagged = 1

        # then calculate the value of each word
        try:
            if postagged == 0:
                value = word[0].pos_score() - word[0].neg_score()
            else:
                value = 0

        except:
            continue
        # store the value for this word
        swnvalues.append((w[0],value))

    return swnvalues


def swnsum(swnlist):
    """
    given list of (word,swnvalues), return the normalized swn value of the text

    a positive value means more good things are said than bad things
    a negative value means more bad than good
    each company's reviews can be compared with other review set values
    """
    swnsum = 0.0
    for i in swnlist:
        swnsum = swnsum + i[1]

    return (swnsum / len(swnlist))


In [24]:
sentimentScores = []
import pandas as pd
data=pd.read_csv('C:/Shiva/Projects/GS_fault_Text_analytics/Medallia/Data/2016-18 All Medallia Surveys Aug 31_2.csv',encoding="ISO-8859-1")


In [25]:
data.head(1)

,Account Name,Account Code,World Key,Contact Salutation,Contact First Name,Contact Last Name,Contact Job Title,Contact Job Seniority,Country,Contact Phone,Contact Mobile,Contact Email,World Key Owner,World Key Owner Email,Account Owner,Account Owner Email,Survey Invitation Date (GMT),Customer Response Date (GMT),Likelihood to Recommend,Likelihood to Recommend Comment,Translation to English for: Likelihood to Recommend Comment,Professionalism,Speed of response,Ease of contacting the relevant person,Effectiveness of different contact channels,Value added by my account team,Ease of finding information,Support to define solutions,Recommending innovative ideas,Usefulness of information,Speed of response for information and pricing,Clarity of contracts,Clarity of quotes,Ease of ordering process,Lead times,Ability to meet communicated lead times,Managing expectations around timings,Skills of the BT team,Ability to resolve service issues,Communications during resolution of service issues,Clarity of billing,Accuracy of billing,Impact on my business,Reliability,Providing innovative technical solutions,Likelihood to Continue in the Future,Anything Else Comment,Translation to English for: Anything Else Comment,Market Unit,Region / Sector,Sub Region / Sub Sector,Parent Segment,Preferred Language,Survey ID,Survey Status,Contact Last Surveyed,Account ID,Contact ID,How did you contact the customer?,How many times did you try to contact the customer?,What was the outcome of the conversation with the customer?,If a follow up meeting is required what date have you scheduled for the meeting (dd/mm/yyyy)?,After your follow up meeting is an AIP required?,If you selected 'No AIP required' please briefly explain why?,Improvement Plan Reference,Alert description,Current status of alert,Alert creation time,Alert close time,Alert closed by,Alert reopened,BT GS Survey Format,Account Team,Understanding your business needs,Ordering your solution,Delivering your solution,Day to day operations,Products and Services,id
0,AIR CHINA - China,70558-CN,WK03834,Mr.,Ziyu,Wang,IT Manager,4. Managers/Executives,China,NaN,NaN,wangziyu@airchina.com,Christina Chen,christina.chen@bt.com,Christina Chen,christina.chen@bt.com,00:00.9,17:54.0,6,"Because in recent years with BT's cooperation,...","Because in recent years with BT's cooperation,...",8.0,8.0,10.0,10.0,9.0,3.0,7.0,7.0,5.0,5.0,7.0,6.0,8.0,7.0,7.0,8.0,6.0,7.0,7.0,8.0,7.0,6.0,7.0,6.0,5.0,NaN,NaN,AMEA,Asia Pac,NaN,Global,Chinese,952424,Completed,NaN,00158000002vFfSAAU,0035800000JpsgKAAR,Phone,1,Available - Follow Up Meeting Scheduled,"August 24th, 2017",No,General issue will be sorted out with customer...,NaN,Accounts likelihood to recommend score 0-6,Closed,18:16.4,28:58.2,Christina Chen (606039565),0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [26]:
type(data)

pandas.core.frame.DataFrame

# Selecting text column

In [27]:
text=data['Likelihood to Recommend Comment']
text=pd.DataFrame(text)


# Encoding

In [28]:
# encoding=utf8
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [29]:
from nltk import word_tokenize
from nltk.corpus import stopwords

# Cleaning text

In [30]:
stop = set(stopwords.words('english'))

In [31]:
def clean(doc):
    doc = " ".join([i.replace('*', '') for i in doc.lower().split()])
    doc = " ".join([i.replace(':', ' ') for i in doc.split()])
    doc = " ".join([i.replace('.', ' ') for i in doc.split()])
    doc = " ".join([i.replace('=', '') for i in doc.split()])
    doc = " ".join([i.replace('/', ' ') for i in doc.split()])
    doc = " ".join([i.replace(')', ' ') for i in doc.split()])
    doc = " ".join([i.replace('(', ' ') for i in doc.split()])
    doc = " ".join([i.replace('"', ' ') for i in doc.split()])
    doc = " ".join([i.replace('-', ' ') for i in doc.split()])
    doc = " ".join([i.replace('_', ' ') for i in doc.split()])
    doc = " ".join([i for i in doc.split() if not i.isdigit()])
    doc = " ".join([i for i in doc.split() if i.isalpha()])
    doc = " ".join([i for i in doc.split() if i not in stop])
    return doc

In [32]:
doc_clean = [clean(doc).split() for doc in text]

In [33]:
doc_clean = [" ".join(i) for i in doc_clean]

In [34]:
text.to_csv('C:/Shiva/Projects/GS_fault_Text_analytics/Medallia/Data/trainData_text_only.csv')

# Sentiment Scoring

In [103]:
sentimentScores = []
with open('C:/Shiva/Projects/GS_fault_Text_analytics/Medallia/Data/trainData_text_only.csv') as f:
    next(f)
    for line in f:
        tokens = line.split()
        stopwords = nltk.corpus.stopwords.words('english')
        parsed = [w.lower() for w in tokens if w.isalpha()]
        parsed = [w for w in parsed if w not in stopwords]
        poslist = nltk.pos_tag(parsed)
        for i in range(len(poslist)):
            if 'VB' in poslist[i][1]:
                poslist[i] = (nltk.stem.WordNetLemmatizer().lemmatize(poslist[i][0], 'v'),'VB')
            parsed[i] = poslist[i][0]
        #textlist = analtext.parser()
        swnlist = swn(parsed)
        #value = swnsum(swnlist)
        swnsum = 0.0
        for i in swnlist:
            swnsum = swnsum + i[1]
        #sentimentScores.append(swnsum / len(swnlist))
        if len(swnlist) == 0:
            value=0
            sentimentScores.append(value)
        else:
            value = swnsum /len(swnlist)
            sentimentScores.append(value)
        #print('Senti-wordnet value: %.7f' % (value * 100))


In [104]:
s = pd.Series(sentimentScores)
result = pd.concat([data, s], axis=1)
result


,Account Name,Account Code,World Key,Contact Salutation,Contact First Name,Contact Last Name,Contact Job Title,Contact Job Seniority,Country,Contact Phone,Contact Mobile,Contact Email,World Key Owner,World Key Owner Email,Account Owner,Account Owner Email,Survey Invitation Date (GMT),Customer Response Date (GMT),Likelihood to Recommend,Likelihood to Recommend Comment,Translation to English for: Likelihood to Recommend Comment,Professionalism,Speed of response,Ease of contacting the relevant person,Effectiveness of different contact channels,Value added by my account team,Ease of finding information,Support to define solutions,Recommending innovative ideas,Usefulness of information,Speed of response for information and pricing,Clarity of contracts,Clarity of quotes,Ease of ordering process,Lead times,Ability to meet communicated lead times,Managing expectations around timings,Skills of the BT team,Ability to resolve service issues,Communications during resolution of service issues,Clarity of billing,Accuracy of billing,Impact on my business,Reliability,Providing innovative technical solutions,Likelihood to Continue in the Future,Anything Else Comment,Translation to English for: Anything Else Comment,Market Unit,Region / Sector,Sub Region / Sub Sector,Parent Segment,Preferred Language,Survey ID,Survey Status,Contact Last Surveyed,Account ID,Contact ID,How did you contact the customer?,How many times did you try to contact the customer?,What was the outcome of the conversation with the customer?,If a follow up meeting is required what date have you scheduled for the meeting (dd/mm/yyyy)?,After your follow up meeting is an AIP required?,If you selected 'No AIP required' please briefly explain why?,Improvement Plan Reference,Alert description,Current status of alert,Alert creation time,Alert close time,Alert closed by,Alert reopened,BT GS Survey Format,Account Team,Understanding your business needs,Ordering your solution,Delivering your solution,Day to day operations,Products and Services,id,0
0,AIR CHINA - China,70558-CN,WK03834,Mr.,Ziyu,Wang,IT Manager,4. Managers/Executives,China,NaN,NaN,wangziyu@airchina.com,Christina Chen,christina.chen@bt.com,Christina Chen,christina.chen@bt.com,00:00.9,17:54.0,6.0,"Because in recent years with BT's cooperation,...","Because in recent years with BT's cooperation,...",8.0,8.0,10.0,10.0,9.0,3.0,7.0,7.0,5.0,5.0,7.0,6.0,8.0,7.0,7.0,8.0,6.0,7.0,7.0,8.0,7.0,6.0,7.0,6.0,5.0,NaN,NaN,AMEA,Asia Pac,NaN,Global,Chinese,952424.0,Completed,NaN,00158000002vFfSAAU,0035800000JpsgKAAR,Phone,1,Available - Follow Up Meeting Scheduled,"August 24th, 2017",No,General issue will be sorted out with customer...,NaN,Accounts likelihood to recommend score 0-6,Closed,18:16.4,28:58.2,Christina Chen (606039565),0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-0.041667
1,NIELSEN COMPANY - USA,52084-US,WK10527,Mr.,Kevin,Young,Global Infrastructure,4. Managers/Executives,United States,(813) 362-7647,(813) 362-7647,kevin.young@neilsen.com,Lynn Roemke,lynn.roemke@bt.com,Lynn Roemke,lynn.roemke@bt.com,00:00.9,54:16.0,10.0,I enjoy the interaction with the technical and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Americas,US&C-EAST,None,Top,English,952468.0,Completed,NaN,00158000002vHRHAA2,00358000006qwlQAAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Accounts likelihood to recommend score 9-10,Closed,54:47.1,59:46.1,Craig LaCava (601415869),0.0,Short,10.0,10.0,NaN,NaN,NaN,10.0,2.0,0.041667
2,ALLIANDER - Netherlands,88286-NL,WK00384,Mr.,Arjan,Olde Damink,Director Alliander Telecom,3. Senior Managers,Netherlands,NaN,621150438,arjan.oldedamink@utilityconnect.nl,Jeroen de Vos,jeroen.devos@bt.com,Jeroen de Vos,jeroen.devos@bt.com,00:00.9,45:33.0,2.0,The reason for this low score is that service ...,The reason for this low score is that service ...,1.0,3.0,3.0,3.0,1.0,NaN,NaN,NaN,NaN,2.0,8.0,8.0,8.0,1.0,1.0,1.0,3.0,7.0,NaN,7.0,1.0,7.0,7.0,NaN,2.0,2017 is so far a year in which the quality of .

In [106]:
result.to_csv('C:/Shiva/Projects/GS_fault_Text_analytics/Medallia/Data/trainData_result.csv')

# Word Cloud

In [2]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt


ImportError: No module named wordcloud

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
cv = CountVectorizer(min_df=0, charset_error="ignore",                                               
                         stop_words="english", max_features=200)
counts = cv.fit_transform([text]).toarray().ravel()                                                  
words = np.array(cv.get_feature_names()) 
# normalize                                                                                                                                             
counts = counts / float(counts.max())


TypeError: __init__() got an unexpected keyword argument 'charset_error'